Let’s walk step-by-step to build a practice AI agent. We'll build a simple customer support agent using OpenAI (or Azure OpenAI), LangChain, and Python, which can:

Greet users

Answer basic product FAQs

Escalate complex issues to a human

Use memory to remember previous conversation

✅ Step 1: Define the Agent's Purpose
We'll build an AI agent that mimics a support assistant for an e-commerce site.

Capabilities:

Handles greetings and small talk

Answers product FAQs from a knowledge base (e.g., shipping, returns)

Escalates to human for unknown questions

Remembers name of user if asked

In [1]:
!pip install langchain openai python-dotenv

You should consider upgrading via the '/Users/samit.manna/Documents/Passion Projects/generative-ai-for-beginners/venv/bin/python3 -m pip install --upgrade pip' command.


In [10]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

In [32]:
load_dotenv()

True

In [25]:
# Simulate a knowledge base
def get_faq_answer(question: str) -> str:
    faqs = {
        "shipping": "We offer free shipping for orders over ₹500. Delivery time is 3-5 business days.",
        "return": "Returns are accepted within 15 days. Please ensure the product is unused.",
        "warranty": "All electronics come with a 1-year warranty.",
    }
    for key in faqs:
        if key in question.lower():
            return faqs[key]
    return "Sorry, I don't know the answer to that."

In [38]:
# Setup LLM with AzureChatOpenAI
llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    openai_api_version="2023-12-01-preview",
    temperature=0.7
)

faq_tool = Tool(
    name="FAQTool",
    func=get_faq_answer,
    description="Use this to answer questions about shipping, returns, and warranties."
)

memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    tools=[faq_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

In [37]:
# Chat loop
if __name__ == "__main__":
    print("🛍️ Welcome to ShopSmart Support! Type 'exit' to leave.\n")
    while True:
        user_input = input("🧑 You: ")
        if user_input.lower() == "exit":
            print("🤖 Agent: Thank you for chatting. Have a great day!")
            break
        try:
            response = agent.run(user_input)
            print("🤖 Agent:", response)
        except Exception as e:
            print("⚠️ Error:", e)

🛍️ Welcome to ShopSmart Support! Type 'exit' to leave.



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No  
AI: Hi there! How can I assist you today?

> Finished chain.
🤖 Agent: Hi there! How can I assist you today?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: FAQTool  
Action Input: What is your shipping policy?  
Observation: We offer free shipping for orders over ₹500. Delivery time is 3-5 business days.
Thought:AI: We offer free shipping for orders over ₹500. Delivery typically takes 3-5 business days. If you have any more questions about shipping or anything else, feel free to ask!

> Finished chain.
🤖 Agent: We offer free shipping for orders over ₹500. Delivery typically takes 3-5 business days. If you have any more questions about shipping or anything else, feel free to ask!


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: FAQTool  
Action Input: How long do returns take?